In [1]:
id: str
spektrix_event_ids: str

dates = ['2025-01-10 18:30:00', '2025-01-11 18:30:00']


In [2]:
from datetime import datetime

import petl as etl

from utils.process.event import Tickets

from utils.paths import PUBLISHED, SITE

In [3]:
OUTPUT_DIR = SITE / 'insights' / id / '_data'
TICKETS_DIR = OUTPUT_DIR / 'tickets'
TICKETS_DIR.mkdir(exist_ok=True, parents=True)

In [4]:
event_times = [datetime.fromisoformat(d) for d in dates]

In [5]:
if spektrix_event_ids:
    tickets = Tickets(spektrix_event_ids.split())
    tickets.detailed().tocsv(TICKETS_DIR / 'by_geography.csv')
    tickets.summarise().tocsv(TICKETS_DIR / 'summary.csv')